In [155]:
from pathlib import Path
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime

#use concat to merge all of files together
list_stacked = pd.DataFrame() 
current_path = Path()/'CSV'
for folder in current_path.absolute().glob('*'):
    for file in folder.glob('H_lvr_land_A.csv'):
        data = pd.read_csv(file)[1:]
        list_stacked = pd.concat([list_stacked, data])

#----------------------------------------------------------------------------------------------------------------------------
#Originally, there are 33 columns in the file; select 18 columns which need to be analyzed
data = list_stacked[['鄉鎮市區', '土地位置建物門牌', '土地移轉總面積平方公尺', '交易年月日', '總樓層數', '建物型態', '建物移轉總面積平方公尺', '總價元',
                     '單價元平方公尺', '主建物面積', '電梯']]


#----------------------------------------------------------------------------------------------------------------------------
#坪 is the most popular unit that Taiwanese to measure the property, so we need to transfer from meter squared to 坪
data = data.copy()
data = data[data['鄉鎮市區'].notna()]
data['土地移轉總面積坪'] = data['土地移轉總面積平方公尺'].astype(float)*0.3025
data['建物移轉總面積坪'] = data['建物移轉總面積平方公尺'].astype(float)*0.3025
data['單價元坪'] = data['單價元平方公尺'].astype(float)*0.3025
data['主建物面積'] = data['主建物面積'].astype(float)*0.3025
data['總價元'] = data['總價元'].astype(float)
data['交易年月日'] = pd.to_datetime((data['交易年月日'].astype(int)+19110000).astype(str), format = '%Y%m%d', errors='coerce') #轉換成日期型態
condition = (data['交易年月日'] >= '2020-10-01') & (data['交易年月日'] <= '2023-09-30') # 篩選資料，只篩選過去36個月的資料
data = data[condition]


# # #----------------------------------------------------------------------------------------------------------------------------

data = data.where(pd.notnull(data), None)
data = data.where(pd.notnull(data), None)
data

# # # #----------------------------------------------------------------------------------------------------------------------------
import pymysql as MySQLdb

try:
    # 開啟資料庫連接
    project = MySQLdb.connect(host="localhost",     # 主機名稱
                           user="root",          # 帳號
                           password="Ff301128",  # 密碼
                           database = "project",  # 資料庫
                           port=3306,            # port
                           charset="utf8")      # 資料庫編碼
    
    # 使用cursor()方法操作資料庫
    cursor = project.cursor()
    
    # 將資料data寫到資料庫中
    try:
        for i in range(len(data)):
            sql = """INSERT INTO Data_20231211 (鄉鎮市區, 土地位置建物門牌, 土地移轉總面積平方公尺, 交易年月日, 總樓層數, 建物型態,
       建物移轉總面積平方公尺, 總價元, 單價元平方公尺, 主建物面積, 電梯, 土地移轉總面積坪, 建物移轉總面積坪,
       單價元坪)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            var = (data.iloc[i, 0], data.iloc[i, 1], data.iloc[i, 2], data.iloc[i, 3], data.iloc[i, 4], 
                   data.iloc[i, 5], data.iloc[i, 6], data.iloc[i, 7], data.iloc[i, 8], data.iloc[i, 9],
                   data.iloc[i, 10], data.iloc[i, 11], data.iloc[i, 12], data.iloc[i, 13])#, data.iloc[i, 14],
                   #data.iloc[i, 15], data.iloc[i, 16], data.iloc[i, 17], data.iloc[i, 18], data.iloc[i, 19],
                   #data.iloc[i, 20], data.iloc[i, 21], data.iloc[i, 22], data.iloc[i, 23], data.iloc[i, 24],
                   #data.iloc[i, 25], data.iloc[i, 26])
            print("Executing SQL:", sql % var)  # Print the SQL query
            cursor.execute(sql, var)
            
        project.commit()        # 提交資料
        
        print("資料寫入完成")
        
        cursor.close()
        project.close()
        
    except Exception as e:
        print("錯誤訊息：", e)
 
except Exception as e:
    print("資料庫連接失敗：", e)
    
finally:
    print("資料庫連線結束")



# #change date of purchase from object type to int and create year, quarter, month and date
# data1 = data.copy()
# data1['交易年月日'] = pd.to_datetime((data['交易年月日'].astype(int)+19110000).astype(str), format = '%Y%m%d', errors='coerce')
# # # data['Year'] = data['交易年月日'].dt.year.fillna(0).astype(int)
# # # data['Quarter'] = data['交易年月日'].dt.quarter.fillna(0).astype(int)
# # # data['Month'] = data['交易年月日'].dt.month.fillna(0).astype(int)
# # # data['Day'] = data['交易年月日'].dt.day.fillna(0).astype(int)


Executing SQL: INSERT INTO Data_20231211 (鄉鎮市區, 土地位置建物門牌, 土地移轉總面積平方公尺, 交易年月日, 總樓層數, 建物型態,
       建物移轉總面積平方公尺, 總價元, 單價元平方公尺, 主建物面積, 電梯, 土地移轉總面積坪, 建物移轉總面積坪,
       單價元坪)
                VALUES (大園區, 青埔段122地號, 211.46, 2023-06-01 00:00:00, None, 其他, 0.0, 10780049.0, 50979, 0.0, 無, 63.96665, 0.0, 15421.1475)
錯誤訊息： (1054, "Unknown column '鄉鎮市區' in 'field list'")
資料庫連線結束
